# 07. lab07-2_Meet MNIST dataset

[MNIST DATA SET](http://yann.lecun.com/exdb/mnist/)

MNIST DATA :
![lec07_6](../../img/lec07_6.png)
X -> Input data : 28 * 27 * 1 데이터로, 총 784개의 x features
Y -> Output data : 0 ~ 9 사이의 10가지 class 의 output  값 (  one-hot incoding 으로 출력)

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
from torch import nn, optim
import numpy as np
import matplotlib.pyplot as plt
import random
import torchvision.datasets as datasets
import torchvision.transforms as transforms
import torch.nn.functional as F

In [2]:
#MNIST data
mnist_trainset = datasets.MNIST(root = './data',train =True,transform=transforms.ToTensor(),download=True)
mnist_testset = datasets.MNIST(root = './data', train = False, transform=transforms.ToTensor(),download=True)

In [3]:
# hyper parameters
training_epochs = 15
batch_size = 100

# Data Loader (Input Pipeline)
train_loader = DataLoader(dataset=mnist_trainset,
                                           batch_size=batch_size,
                                           shuffle=True)

test_loader = DataLoader(dataset=mnist_testset,
                                          batch_size=batch_size,
                                          shuffle=False)

In [17]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.linear = nn.Linear(784, 10,bias = True) # 2 in and 1 out
        
    def forward(self, x):
        x = x.view(-1, 784)
        y_pred = self.linear(x)
        F.softmax(y_pred,dim = 1)
        return y_pred
    
model = Model()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [18]:
for epoch in range(training_epochs):

    avg_cost = 0
    total_batch = int(len(mnist_trainset) / batch_size)
    model.train()
    
    for idx, (batch_xs, batch_ys) in enumerate(train_loader):
        optimizer.zero_grad()
        y_pred = model(batch_xs)
        loss = criterion(y_pred, batch_ys)
        loss.backward()
        avg_cost += loss / total_batch
    print('Train Epoch: {}\tLoss: {:.6f}'.format(epoch, avg_cost))
    optimizer.step()

#         if idx % 10 == 0:
#             print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
#                 epoch, idx * len(data), len(train_loader.dataset),
#                 100. * idx / len(train_loader), loss.data[0]))

Train Epoch: 0	Loss: 2.324386
Train Epoch: 1	Loss: 1.978600
Train Epoch: 2	Loss: 1.695961
Train Epoch: 3	Loss: 1.461045
Train Epoch: 4	Loss: 1.256177
Train Epoch: 5	Loss: 1.106402
Train Epoch: 6	Loss: 1.008929
Train Epoch: 7	Loss: 0.932672
Train Epoch: 8	Loss: 0.867257
Train Epoch: 9	Loss: 0.807285
Train Epoch: 10	Loss: 0.752935
Train Epoch: 11	Loss: 0.716198
Train Epoch: 12	Loss: 0.684900
Train Epoch: 13	Loss: 0.661016
Train Epoch: 14	Loss: 0.635878


### Test

In [21]:
model.eval()
test_loss = 0
correct = 0
for data, target in test_loader:
    data, target = Variable(data), Variable(target)
    output = model(data)
    # sum up batch loss
    test_loss += criterion(output, target).item()
    # get the index of the max log-probability
    pred = output.data.max(1, keepdim=True)[1]
    correct += pred.eq(target.data.view_as(pred)).cpu().sum()


In [22]:
test_loss /= len(test_loader.dataset)
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
    test_loss, correct, len(test_loader.dataset),
    100. * correct / len(test_loader.dataset)))


Test set: Average loss: 0.0058, Accuracy: 8291/10000 (82%)

